In [1]:
import sys
import os

# Make sure your project root is on PYTHONPATH
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from src.data_loader import ShanghaiTechDataModule
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from src.utils import get_model, get_device


## RESNET 

In [3]:
# ====== Part A evaluation (ResNet-50 density-map backbone only) ======
device = "mps"
data_module = ShanghaiTechDataModule(
    data_folder="../data/ShanghaiTech",
    part="part_A",
    validation_split=0.1,
    sigma=5,
    return_count=False,
    batch_size=1,
    num_workers=4,
    input_size=(384, 384),
    density_map_size=(192, 192),
    device=device
)
val_loader_A = data_module.test_dataloader()

model = get_model("resnet50",
                  cpt="/Users/vittorio/Projects/uni/CV/CV-crowd-flow-estimation-/models/pth/part_A_resnet50_15.pth",
                  device=device,
                  )[0]
model.eval()

# Evaluate
total_mae = 0.0
total_mse = 0.0

with torch.no_grad():
    for img, gt_density in val_loader_A:
        img        = img.to(device)       # B×3×H×W
        gt_density = gt_density.to(device)  # B×1×H×W

        # True count
        gt_count = gt_density.sum(dim=(1,2,3))  # B

        # Predicted density and count
        pred_density = model(img)               # B×1×H×W
        pred_count   = pred_density.sum(dim=(1,2,3))

        # Accumulate
        total_mae += torch.abs(pred_count - gt_count).sum().item()
        total_mse += ((pred_count - gt_count)**2).sum().item()

N    = len(val_loader_A)
mae  = total_mae / N
rmse = (total_mse / N)**0.5

print(f"Part A — resnet34: MAE = {mae:.3f}, RMSE = {rmse:.3f}")


Loading checkpoint '/Users/vittorio/Projects/uni/CV/CV-crowd-flow-estimation-/models/pth/part_A_resnet50_15.pth'
Checkpoint loaded successfully
Part A — resnet34: MAE = 138.598, RMSE = 200.679


# VGG encoder

In [ ]:
model = get_model("vgg19_bn", pretrained=True, cpt="../models/pth/part_A_vgg19.pth")[0]

# Evaluate
total_mae = 0.0
total_mse = 0.0

with torch.no_grad():
    for img, gt_density in val_loader_A:
        img        = img.to(device)       # B×3×H×W
        gt_density = gt_density.to(device)  # B×1×H×W

        # True count
        gt_count = gt_density.sum(dim=(1,2,3))  # B

        # Predicted density and count
        pred_density = model(img)               # B×1×H×W
        pred_count   = pred_density.sum(dim=(1,2,3))

        # Accumulate
        total_mae += torch.abs(pred_count - gt_count).sum().item()
        total_mse += ((pred_count - gt_count)**2).sum().item()

N    = len(val_loader_A)
mae  = total_mae / N
rmse = (total_mse / N)**0.5

print(f"Part A — VGG19bn: MAE = {mae:.3f}, RMSE = {rmse:.3f}")